In [ ]:

from openai import AzureOpenAI
import os
import requests
from PIL import Image
import json

import prompt_generator_for_gpt

In [ ]:
fname_DALLE_KEY = 'DALLE_TOKEN.txt' # TODO: change to your own API key
if os.path.isfile(fname_DALLE_KEY):
    with open(fname_DALLE_KEY, 'r') as fh:
        AZURE_DALLE_API_KEY = fh.read()
else:
    print('Error: AZURE_DALLE_API_KEY file not found')
AZURE_OPENAI_ENDPOINT = 'https://rhaetian-poppy-sweden.openai.azure.com/'

client = AzureOpenAI(
    api_version="2023-12-01-preview",  
    api_key=AZURE_DALLE_API_KEY, #os.environ["AZURE_OPENAI_API_KEY"],  
    azure_endpoint=AZURE_OPENAI_ENDPOINT # os.environ['AZURE_OPENAI_ENDPOINT']
)

prompt_str = prompt_generator_for_gpt.prompt_string()

result = client.images.generate(
    model="rhaetian-poppy-dalle3", # the name of your DALL-E 3 deployment
    prompt=prompt_str,
    n=1
)

json_response = json.loads(result.model_dump_json())

# Set the directory for the stored image
image_dir = os.path.join(os.curdir, 'images')

# If the directory doesn't exist, create it
if not os.path.isdir(image_dir):
    os.mkdir(image_dir)

# Initialize the image path (note the filetype should be png)
image_path = os.path.join(image_dir, 'generated_image.png')

# Retrieve the generated image
image_url = json_response["data"][0]["url"]  # extract image URL from response
generated_image = requests.get(image_url).content  # download the image
with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

# Display the image in the default image viewer
image = Image.open(image_path)
image.show()

In [ ]:
prompt_str